In [ ]:
# This Python 3 environment comes with many splitterpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgbm

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
y = train['claim']
train.drop(columns=['claim', 'id'], inplace=True)

## Use only 20% of dataset, stratified represent both 0s and 1s in claim column

In [ ]:
splitter = StratifiedShuffleSplit( train_size=0.2)
train_idx, test_idx = next(splitter.split(train, y))
X_train, y_train = train.iloc[train_idx], y.iloc[train_idx]

## Missing values: experimental IterativeImputer to fill all features with BayesianRidge with another features usage

In [ ]:
imputer = IterativeImputer(max_iter=10, random_state=123)
X_train = imputer.fit_transform(X_train)

## Simple classifier

In [ ]:
clf = lgbm.LGBMClassifier(n_estimators=10000, n_jobs=-1, learning_rate=0.01)
x_train, x_valid, y_train, y_valid = train_test_split(X_train, y_train.values, shuffle=True)

In [ ]:
clf.fit(
    x_train, y_train,
    eval_set = [(x_valid, y_valid)],
    verbose=500
)

## Read test dataset and predict values

In [ ]:
test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv', index_col='id')


In [ ]:
pd.DataFrame({'id': test.index, 'claim': clf.predict_proba(imputer.transform(test))[:, 1]}).to_csv('submission.csv', index=False)